# La gestion de l’encodage avec Python 3


Python 3 introduit une révolution dans la gestion de l’encodage des caractères, au prix d’une confusion troublante pour qui vient de Python 2 :
- en Python 2, les chaînes de caractères (type `str`) sont par défaut encodées
- en Python 3, elles sont décodées

Type par défaut d’une chaîne de caractères décodée en Python 3 :

In [ ]:
# Chaîne décodée (type 'str')
type('chaîne de caractères')

Une chaîne encodée ne doit contenir que des caractères ASCII :

In [ ]:
# Chaîne encodée (type 'bytes')
print(b'chaîne de caractères')

Pour la manipuler sous forme d’octets, il faut alors l’encoder en choisissant un encodage :

In [ ]:
print('chaîne de caractères'.encode('utf8'))
print(bytes('chaîne de caractères', 'latin1'))

À l’inverse, une liste d’octets en entrée doit être décodée pour s’afficher :

In [ ]:
chaine = 'chaîne de caractères'.encode('utf8')
print(chaine.decode('utf8'))

Il convient donc de connaître l’encodage utilisé, sinon le décodage se passe mal :

In [ ]:
print(chaine.decode('latin1'))
print(chaine.decode('mac-roman'))
print(chaine.decode('cp855'))

En bref, lorsque Python 3 manipule du texte, il offre toujours un affichage unicode. Les ennuis commencent lorsque l’on veut manipuler des données binaires :

In [ ]:
a = 'chaîne de caractères'
b = 'chaîne de caractères'.encode('latin1')
print(a[3])
print(b[3])

L’accès atomique à des données binaires se fait donc sous forme d’octet. Pour savoir quel caractère unicode se cache derrière un entier, on utilise la fonction `chr()` :

In [ ]:
chr(238)

Et pour connaître le numéro unicode d’un caractère décodé, on utilise la fonction `ord()` :

In [ ]:
letters = ['ε', 'ܬ']
[ord(l) for l in letters]

## Cas pratique

En japonais, le mot *paix* s’orthographie *平和* (*heiwa*). Il est constitué de deux caractères :
- *平*
- *和*

Leurs numéros d’ordre Uncicode s’obtiennent facilement :

In [ ]:
[ord(c) for c in '平和']

Ces numéros sont fournis en base 10 (décimale). Or, le langage informatique repose sur une base 2 (binaire).

Si le premier idéogramme peut se représenter sur 5 signes allant de 0 à 9 (24179), il est impossible de le représenter sur un octet (8 signes allant de 0 à 1).

Comme il faut plus d’un octet pour le représenter, toute conversion est impossible :

In [ ]:
print(b'平')

Pour rendre la conversion possible, on doit d’abord fournir un encodage adéquat :

In [ ]:
# Jeu de caratères latin1 ne prévoit pas le signe 平
c = '平'.encode('latin1')

In [ ]:
# UTF-8 permet d'encoder le signe
c = '平'.encode('utf8')

**Attention !** `c` est désormais une donnée binaire, représentée sous forme d’octets !

In [ ]:
# Nombre d'octets nécessaires à l'encodage de l'idéogramme *平* en UTF-8
print(len(c))
# Affichage de tous les octets (en base 16)
print(c)
# Affichage de tous les octets (en base 10)
[print(octet) for octet in c]
# Et en base 2
[print(bin(octet)) for octet in c]
# Grâce à une f-String
[f'{octet:b}' for octet in c]

## Unicode et Python

**Rappel :** toute chaîne affichée par Python3 est compatible Unicode

Module `unicodedata` avec des méthodes pour manipuler les caractères :
- `lookup()` : rechercher un caractère par son nom
- `decimal()` : obtenir la valeur décimale du caractère
- `normalize()` : obtenir une version normalisée du caractère
- …

Documentation sur le module :  
[https://docs.python.org/3/library/unicodedata.html](https://docs.python.org/3/library/unicodedata.html)

### Ordre lexicographique

Une opération commune en analyse textuelle consiste à trier des éléments (*i.e.* des mots) par ordre alphabétique :

In [ ]:
words = ['lettre', 'apostrophe']
words.sort()
print(words)

Le premier écueil se manifeste dès que des majuscules s’invitent dans la liste :

In [ ]:
words = ['lettre', 'apostrophe', 'Ellipse']
words.sort()
print(words)

C’est ici qu’intervient le paramètre `key` pour transmettre une fonction à chaque élément de la liste, avant toute comparaison :

In [ ]:
words = ['lettre', 'apostrophe', 'Ellipse']
# Each word is converted to lowercase before sorting
words.sort(key=str.lower)
print(words)

Méthode plutôt efficace et suffisante pour l’anglais, mais qui montre ses limites dès qu’entrent en jeu des accents :

In [ ]:
words = ['lettre', 'apostrophe', 'Ellipse', 'à']
words.sort(key=str.lower)
print(words)

L’un des remèdes possibles consiste à supprimer les accents des chaînes de caractères.

Plutôt que de prévoir tous les cas possibles (*Si le caractère se trouve parmi \[à, â, ä\], alors le transformer en "a", et sinon, si le caractère est un parmi \[éè\]…*), la fonction `normalize()` du module `unicodedata` permet d’effectuer ce calcul plus simplement.

En résumé, la fonction décompose un caractère en ses différents composants. Exemple avec la "ligature minuscule latine fi" :

In [ ]:
import unicodedata
fi = unicodedata.lookup('Latin Small Ligature Fi')
components = unicodedata.normalize('NFKD', fi)
print(f'Le caractère "{fi}" est composé de : {[c for c in components]}')

Appliquée à notre collection de mots, les caractères accentués ne sont plus relégués à la fin :

In [ ]:
import unicodedata
words = ['lettre', 'apostrophe', 'Ellipse', 'à']
words.sort(key=lambda this:unicodedata.normalize('NFKD', this))
print(words)

En revanche, elle soulève d’autres problèmes :
- perte de la distinction majuscules/minuscules
- perte du tri par nombre de caractères

Plutôt que de transmettre une fonction anonyme au paramètre `key`, autant définir une fonction utilisateur :

In [ ]:
import unicodedata

def comparison(word):
    """Removes the accent marks on every character in a word.
    
    Keyword argument:
    word -- the word to analyze
    """
    norm = ""
    word = word.lower()
    for ch in word:
        components = unicodedata.normalize('NFKD', ch)
        norm += components[0]
    return norm

words = ['lettre', 'apostrophe', 'Ellipse', 'à']
words.sort(key=comparison)
print(words)